In [6]:
#%%writefile get_ruten.py
#%load get_bookprice.py
#%run get_bookprice.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
from django.db.models import Q
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq
#
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
#
import requests
import urllib.parse
import pandas as pd
import numpy as np
import random
import re
import json
import csv
#
from get_proxy import get_proxy
#from get_mollie import get_mollie
#
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'wtb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
#from mainsite.models import Bookinfo,Bookprice,Store
#from dict_stores import url_qs

#
def get_ruten(isbn):
    url_api1="https://rtapi.ruten.com.tw/api/search/v2/index.php/core/prod?sort=prc%2Fac&q="
    url_api2="https://rtapi.ruten.com.tw/api/search/v2/index.php/m/core/prod?sort=prc%2Fac&q="
    url_apis=[url_api1,url_api2]
    url_q=random.choice(url_apis)+isbn
    #(1)先抓價格最低的店內碼______________________
    #
    fake_header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )    
    UA=fake_header.generate()  
    ippo=get_proxy(which="OK",now=True)
    proxies={
            "http": "http://"+ippo,
            #"https": "http://"+ippo
            }
    try:
        r = requests.get(url_q, 
                         headers=UA,
                         proxies=proxies,
                         #allow_redirects=False,
                         timeout=20)    
        r.encoding='utf8'        
        #
        ans_dict=json.loads(r.text)
        r.close()
        #
        #print(ans_dict)
        TotalRows=ans_dict['TotalRows']
        if TotalRows>0:
            rt_id=ans_dict['Rows'][0]['Id']
        else:
            return []
        print(rt_id)
        #(2)抓商品頁的售價______________________
        url_book="https://goods.ruten.com.tw/item/show?"+rt_id  
        url_book_m="https://m.ruten.com.tw/goods/show.php?g="+rt_id
        r = requests.get(url_book_m, 
                         headers=UA,
                         proxies=proxies,
                         #allow_redirects=False,
                         timeout=30) 
        r.encoding='utf8'
        #
        doc=pq(r.text)
        r.close()
        #print(r.text)
        #
        #desc=doc.find('meta[property="og:description"]').attr('content') #PC版常有亂碼
        desc=doc.find("script[type='application/ld+json']").eq(0).text()
        price_sale=re.search('直購價：([0-9]+?)元',desc).group(1)
        return [price_sale,url_book]
    except Exception as err:
        return []#'err'+str(err)


In [7]:
isbn='9789571371870'
%time get_ruten(isbn)

21903571291291
CPU times: user 33.2 ms, sys: 6.28 ms, total: 39.5 ms
Wall time: 2.26 s


['326', 'https://goods.ruten.com.tw/item/show?21903571291291']